In [18]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "qwerty123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'my-image.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
# html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px', 'height': 'auto'})),
    html.Center(html.B(html.H1('Robert Fjellin Project Two Dashboard'))),
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain Rescue', 'value': 'mountain'},
            {'label': 'Disaster Rescue', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='all',
        labelStyle={'display': 'inline-block', 'margin-right': '15px'},
        style={'textAlign': 'left', 'margin-bottom': '10px'}
    )
]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size=10,               # Records per page
                         page_current=0,             # Default page
                         page_action='native',
                         sort_action='native',
                         sort_mode='multi',          # Allow multiple sorts
                         filter_action='native',     # Default filtering
                         row_selectable='single',    # Only allow one row selected at a time
                         selected_rows=[0],          # Select default row
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                            
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    ])
#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)
# Create empty query dictionary
# Create empty query dictionary
    # Create empty query dictionary
    # Create empty query dictionary
    
    try:
        # Get all data
        all_docs = db.read({})
        if not all_docs:
            return []
            
        # Convert to DataFrame
        df = pd.DataFrame.from_records(all_docs)
        
        # Remove _id column
        if '_id' in df.columns:
            df = df.drop(columns=['_id'])
        
        # Apply breed filters
        if filter_type == 'water':
            # Water rescue breeds
            df = df[df['breed'].str.contains('Labrador|Retriever|Newfoundland|Chesapeake', case=False, na=False)]
            # Water rescue sex
            df = df[df['sex_upon_outcome'] == "Intact Female"]
            
        elif filter_type == 'mountain':
            # Mountain rescue breeds
            df = df[df['breed'].str.contains('German Shepherd|Husky|Malamute|Rottweiler', case=False, na=False)]
            # Mountain rescue sex
            df = df[df['sex_upon_outcome'] == "Intact Male"]
            
        elif filter_type == 'disaster':
            # Disaster rescue breeds
            df = df[df['breed'].str.contains('Doberman|German Shepherd|Golden Retriever|Bloodhound|Rottweiler', case=False, na=False)]
            # Disaster rescue sex
            df = df[df['sex_upon_outcome'] == "Intact Male"]
        
        # Apply age filter if the column exists
        if 'age_upon_outcome_in_weeks' in df.columns:
            if filter_type == 'water' or filter_type == 'mountain':
                df = df[(df['age_upon_outcome_in_weeks'] >= 26) & (df['age_upon_outcome_in_weeks'] <= 156)]
            elif filter_type == 'disaster':
                df = df[(df['age_upon_outcome_in_weeks'] >= 20) & (df['age_upon_outcome_in_weeks'] <= 300)]
        
        # Return the filtered data
        return df.to_dict('records')
    
    except Exception as e:
        print(f"Error: {str(e)}")
        return []

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    if viewData is None:
        return []
    
    # Convert the filtered data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    if dff.empty:
        return html.Div("No data available")
    
    # Create a pie chart showing the distribution of breeds
    chart = px.pie(
        dff, 
        names='breed', 
        title='Animal Breed Distribution',
        color_discrete_sequence=px.colors.qualitative.Pastel
    )
    
    # Improve layout
    chart.update_layout(
        legend=dict(
            orientation="h", 
            yanchor="bottom", 
            y=-1
        ),
        margin=dict(l=20, r=20, t=40, b=20)
    )
    
    return [
        dcc.Graph(figure=chart)
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        selected_columns = []
    
    # Highlight selected columns
    column_style = [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]
    
    # Add additional styling for better readability
    additional_styles = [
        {
            'if': {'row_index': 'odd'},
            'background_color': '#f9f9f9'
        },
        {
            'if': {'state': 'selected'},
            'background_color': '#EEEEEE',
            'border': '1px solid #CCCCCC'
        }
    ]
    
    return column_style + additional_styles


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True, port=29355)


Connection was successful!
Dash app running on http://127.0.0.1:29355/
